### Ideas on generating question-answer pairs over a specific document that can be used to build the Knowledge base

#### Set the Environment Variable

In [1]:
import os  
import json  
import openai
from Utilities.envVars import *

# Set Search Service endpoint, index name, and API key from environment variables
indexName = SearchIndex

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = OpenAiService
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint
davincimodel = OpenAiDavinci


In [2]:
# import os
# import openai
# from Utilities.envVars import *

In [3]:
# openai.api_key = OpenAiKey
# davincimodel= OpenAiDavinci
# davincimodel2= OpenAiDavinci2

In [4]:
# !pip install langchain==0.0.177

#### Import Required Library

In [5]:
# Import required libraries
from langchain.llms.openai import AzureOpenAI, OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import (
    PDFMinerLoader,
    UnstructuredFileLoader,
)
from langchain.chat_models import ChatOpenAI
from langchain.chains import QAGenerationChain

In [6]:
OpenAiChat

'GPT35Turbo'

In [7]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chat_models import AzureChatOpenAI, ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from IPython.display import display, HTML

embeddingModelType = "azureopenai"
temperature = 0.3
tokenLength = 1000

if (embeddingModelType == 'azureopenai'):
        openai.api_type = "azure"
        openai.api_key = OpenAiKey
        openai.api_version = OpenAiVersion
        openai.api_base = openAiEndPoint#f"https://{OpenAiService}.openai.azure.com"

        llm = AzureChatOpenAI(
                openai_api_base=openai.api_base,
                openai_api_version=OpenAiVersion,
                deployment_name=OpenAiDavinci,
                temperature=temperature,
                openai_api_key=OpenAiKey,
                openai_api_type="azure",
                max_tokens=tokenLength,
                model_name=OpenAiDavinci)
        embeddings = OpenAIEmbeddings(deployment=OpenAiEmbedding, chunk_size=1, openai_api_key=OpenAiKey)
        logging.info("LLM Setup done")
# elif embeddingModelType == "openai":
#         openai.api_type = "open_ai"
#         openai.api_base = "https://api.openai.com/v1"
#         openai.api_version = '2020-11-07' 
#         openai.api_key = OpenAiApiKey
#         llm = ChatOpenAI(temperature=temperature,
#         openai_api_key=OpenAiApiKey,
#         model_name="gpt-3.5-turbo",
#         max_tokens=tokenLength)
#         embeddings = OpenAIEmbeddings(openai_api_key=OpenAiApiKey)

In [8]:
llm

AzureChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='Davinci003', temperature=0.3, model_kwargs={}, openai_api_key='555d51f88e9e471b9f417da8b3a42dd6', openai_api_base='https://gpt3tests.openai.azure.com/', openai_organization='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=1000, deployment_name='Davinci003', openai_api_type='azure', openai_api_version='2022-12-01')

In [9]:
OpenAiChat

'GPT35Turbo'

In [10]:
llm,OpenAiChat

(AzureChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='Davinci003', temperature=0.3, model_kwargs={}, openai_api_key='555d51f88e9e471b9f417da8b3a42dd6', openai_api_base='https://gpt3tests.openai.azure.com/', openai_organization='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=1000, deployment_name='Davinci003', openai_api_type='azure', openai_api_version='2022-12-01'),
 'GPT35Turbo')

In [11]:
# Set the file name and the namespace for the index
fileName = "Fabric Get Started.pdf"
fabricGetStartedPath = "Data/PDF/" + fileName
# Load the PDF with Document Loader available from Langchain
loader = PDFMinerLoader(fabricGetStartedPath)
rawDocs = loader.load()


In [12]:
type(rawDocs)

list

In [13]:
print(rawDocs[0].page_content)

Tell us about your PDF experience.

Microsoft Fabric get started
documentation

Microsoft Fabric is a unified platform that can meet your organization's data and
analytics needs. Discover the Fabric shared and platform documentation from this page.

About Microsoft Fabric

ｅ OVERVIEW

What is Fabric?

Fabric terminology

ｂ GET STARTED

Start a Fabric trial

Fabric home navigation

End-to-end tutorials

Context sensitive Help pane

Get started with Fabric items

ｐ CONCEPT

Find items in OneLake data hub

Promote and certify items

ｃ HOW-TO GUIDE

Apply sensitivity labels

Workspaces

ｐ CONCEPT

Fabric workspace

Workspace roles

ｂ GET STARTED

Create a workspace

ｃ HOW-TO GUIDE

Workspace access control

What is Microsoft Fabric?

Article • 05/23/2023

Microsoft Fabric is an all-in-one analytics solution for enterprises that covers everything

from data movement to data science, Real-Time Analytics, and business intelligence. It

offers a comprehensive suite of services, including dat

In [14]:
rawDocs[0].metadata

{'source': 'Data/PDF/Fabric Get Started.pdf'}

In [15]:
fabricGetStartedPath

'Data/PDF/Fabric Get Started.pdf'

In [16]:
# Set the source 
for doc in rawDocs:
    doc.metadata['source'] = fabricGetStartedPath

textSplitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = textSplitter.split_documents(rawDocs)

In [17]:
len(docs),print(docs[1].page_content)

Microsoft Fabric is currently in PREVIEW. This information relates to a prerelease

product that may be substantially modified before it's released. Microsoft makes no

warranties, expressed or implied, with respect to the information provided here.

SaaS foundation

Microsoft Fabric brings together new and existing components from Power BI, Azure

Synapse, and Azure Data Explorer into a single integrated environment. These

components are then presented in various customized user experiences.

Fabric brings together experiences such as Data Engineering, Data Factory, Data Science,

Data Warehouse, Real-Time Analytics, and Power BI onto a shared SaaS foundation. This

integration provides the following advantages:

An extensive range of deeply integrated analytics in the industry.

Shared experiences across experiences that are familiar and easy to learn.

Developers can easily access and reuse all assets.

A unified data lake that allows you to retain the data where it is while using

(58, None)

In [20]:
chain = QAGenerationChain.from_llm(llm)

In [21]:
qa = chain.run(docs[55].page_content)

InvalidRequestError: Resource not found

In [ ]:
qa